In [ ]:
import os
#os.chdir('../../10x_tupro/')
#os.chdir('../../maynard_human_brain_analysis/')
#os.chdir('../../her2_positive_breast_tumors/')

In [ ]:
from plotnine_prism import *
import plotnine as p9
from tqdm import tqdm
import pandas as pd
import numpy as np
import glob
import yaml

In [ ]:
with open("../model_and_dataset_info.yaml", "r") as stream:
    model_and_dataset_info = yaml.safe_load(stream)
model_and_dataset_info

In [ ]:
out_folder = "out_benchmark"

In [ ]:
with open("info.yaml", "r") as stream:
    INFO = yaml.safe_load(stream)
MODEL_R = INFO["MODEL_R"] if INFO["MODEL_R"] else list()
MODEL_PYTHON = INFO["MODEL_PYTHON"] if INFO["MODEL_PYTHON"] else list()
MODEL_FINE_TUNE = INFO["MODEL_FINE_TUNE"] if INFO["MODEL_FINE_TUNE"] else list()

MODELS = [*MODEL_R, *MODEL_PYTHON, *MODEL_FINE_TUNE]
MODELS

In [ ]:
benchmark_files = glob.glob(f"{out_folder}/benchmarks/**/*.log", recursive=True)
benchmark_files[:2]

In [ ]:
data = []
for file in tqdm(benchmark_files):
    rule = file.split("/")[2].split(".log")[0]
    df = pd.read_csv(file, sep="\t")
    df["rule"] = rule
    data.append(df)
data = pd.concat(data)
data

In [ ]:
data["time_min"] = data.s / 60

In [ ]:
measurements = ["time_min", "max_rss", "max_vms", "max_uss", "max_pss","io_in", "io_out", "mean_load", "cpu_time"]

In [ ]:
tab = data.query("rule in @MODELS")
tab["model"] = pd.Categorical(tab["rule"], tab.groupby("rule")["time_min"].agg("median").sort_values().index)
tab["modality"] = tab.model.apply(lambda x: model_and_dataset_info["model_modality"][x.split("_")[0]])

In [ ]:
current_path = !pwd
dataset = model_and_dataset_info["dataset"][current_path[0].split("/")[-1]]
dataset

In [ ]:
print(p9.ggplot(tab, p9.aes("model", "time_min")) 
 + p9.geom_boxplot(p9.aes(fill="modality"), alpha=0.8)
 + p9.theme_bw()
 + p9.coord_flip()
 + scale_fill_prism(palette = "colors")
 + p9.ylab("Min")
 + p9.xlab("Model")
 + p9.ggtitle(f"Runtime on\n{dataset}")
)

In [ ]:
for measurement in measurements: 
    data["rule"] = data["rule"].astype(str)
    data["rule"] = pd.Categorical(data["rule"], data.groupby("rule")[measurement].agg("median").sort_values().index)
    print(p9.ggplot(data, p9.aes("rule", measurement))
             + p9.geom_boxplot()
             + p9.theme_bw()
             + p9.coord_flip()
            )
    

In [ ]:
data.groupby("rule").agg("max")

In [ ]:
os.chdir(f'{out_folder}/logs/')

In [ ]:
info = !reportseff --format "+ReqMem%30,AllocCPUS,jobname"
info[:2]

In [ ]:
columns = info[0].split(" ")
columns = [l for l in columns if len(l)>0]
columns

In [ ]:
summary = []
for line in info[1:]:
    if "--" not in line:
        line = line.split(" ")
        line = [l for l in line if len(l)>0]
        JobID = line[0]
        State = line[1]
        Elapsed = line[2]
        TimeEff = float(line[3].replace("%", ""))
        CPUEff = float(line[4].replace("%", ""))
        MemEff = float(line[5].replace("%", ""))
        ReqMem = line[6]
        ReqMem_G = float(ReqMem.split("G")[0]) if "G" in ReqMem else float(ReqMem.split("M")[0]) * 0.001
        AllocCPUS = int(line[7])
        jobname = line[8]
    
    summary.append([JobID, State, Elapsed, TimeEff, CPUEff, MemEff, ReqMem_G, AllocCPUS, jobname])
summary = pd.DataFrame(summary, columns=columns)
summary

In [ ]:
summary.State.value_counts()

In [ ]:
summary[["State", "JobName"]].value_counts()

In [ ]:
summary = summary.query("State == 'COMPLETED'").copy()
summary["utilised_mem_in_G"] = summary.apply(lambda x: ((x.MemEff) / 100) * x.ReqMem, axis=1)
summary["utilised_CPUS"] = summary.apply(lambda x: ((x.CPUEff) / 100) * x.AllocCPUS, axis=1)
summary

In [ ]:
for col in summary.columns[3:]:
    if col != "JobName":
        tab = summary.copy()
        tab.JobName = pd.Categorical(tab.JobName, tab.groupby("JobName")[col].agg("median").sort_values().index)
        print(p9.ggplot(tab, p9.aes("JobName", col)) 
         + p9.geom_boxplot()
         + p9.theme_bw()
         + p9.coord_flip()
        )